In [1]:
from bs4 import BeautifulSoup as soup
import requests
import pandas as pd
import sys
from io import StringIO

In [6]:
url_pagination = 'https://store.steampowered.com/search/'

r = requests.get(url_pagination)
page_soup = soup(r.content, "html.parser")

containers = page_soup.findAll("a", {"class":"search_result_row ds_collapse_flag "})

filename = "products.csv"
f = open(filename, "w")

headers = "game_title, steam_url\n"
f.write(headers)

for container in containers:
    game_title = container.findAll("span", {"class":"title"})[0].text
    link = container["href"]
    
    print(game_title)
    print(link)
    print("----------------")
    
    f.write(game_title + "," + link + "\n")
    
f.close()

In [18]:
# Get title and links for all steam games
urls = ['https://store.steampowered.com/search/?category1=998&page={}'.format(i) for i in range(1, 1184)]

filename = "products.csv"
f = open(filename, "w", encoding="utf-8")
headers = "game_title\tsteam_url\n"
f.write(headers)

for url in urls:
    r = requests.get(url)
    page_soup = soup(r.content, "html.parser")

    containers = page_soup.findAll("a", {"class":"search_result_row ds_collapse_flag "})

    for container in containers:
        game_title = container.findAll("span", {"class":"title"})[0].text
        link = container["href"]

        #print(game_title)
        #print(link)
        #print("----------------")

        f.write(game_title + "\t" + link + "\n")

f.close()

In [16]:
df_links = pd.read_csv('products.csv', delimiter="\t")
df_links.head()

,game_title,steam_url
0,Insurgency: Sandstorm,https://store.steampowered.com/app/581320/Insu...
1,MONSTER HUNTER: WORLD,https://store.steampowered.com/app/582010/MONS...
2,Counter-Strike: Global Offensive,https://store.steampowered.com/app/730/Counter...
3,Divinity: Original Sin 2 - Definitive Edition,https://store.steampowered.com/app/435150/Divi...
4,Grand Theft Auto V,https://store.steampowered.com/app/271590/Gran...


In [ ]:
# Get tags for each game
cookies = {'birthtime': '568022401'}
df_links = pd.read_csv('products.csv', delimiter="\t")
df_tag = pd.DataFrame()



for link in df_links.iterrows():
    try:
        url = link[1]['steam_url']
        r = requests.get(url, cookies=cookies)
        page_soup = soup(r.content, "html.parser")
        title = page_soup.find('span', {'itemprop':'name'}).text
        tags = page_soup.find("div", {"class":"glance_tags popular_tags"})
        tags = tags.text
        tags = tags.replace('\t', '')
        tags = tags.replace('\r', '')
        tags = tags.replace('\n', ', ')
        tags = tags.replace('+, ', '')
        tags = tags.replace(', , ', '')
        df_tags = pd.DataFrame([tags])
        df_tag = df_tag.append(df_tags)
    except AttributeError:
        title = "skip"
        tags = "N/A"
        df_tags = pd.DataFrame([tags])
        df_tag = df_tag.append(df_tags)


    
df_tag_2 = df_tag.reset_index()
df_tag_final = df_tag_2.drop(['index'], axis=1)

steam_tags = pd.concat([df_links, df_tag_final], axis=1, sort=False)
steam_tags.to_csv('steam_tags', sep="\t", header=['title', 'steam_url', 'steam_tags'])

In [168]:
cookies = {'birthtime': '568022401'}
r = requests.get('https://store.steampowered.com/agecheck/app/359550/', cookies=cookies)
page_soup = soup(r.content, "html.parser")

In [176]:
page_soup.find('span', {'itemprop':'name'}).text

"Tom Clancy's Rainbow Six® Siege"

In [94]:
tags = []
tags = page_soup.find("div", {"class":"glance_tags popular_tags"})
tags = tags.text
tags = tags.replace('\t', '')
tags = tags.replace('\r', '')
tags = tags.replace('\n', ', ')
tags = tags.replace('+, ', '')
tags = tags.replace(', , ', '')
tags= StringIO(tags)
print(tags)
df = pd.read_csv(tags, header=None, sep='|')
df.head()

,0
0,"Massively Multiplayer, MMORPG, Open World, RPG..."


In [143]:
cookies = {'birthtime': '568022401'}
df_links = pd.read_csv('products.csv', delimiter="|")
df_tag = pd.DataFrame()

while True:
    try:
        for link in df_links.iterrows():
            url = link[1]['steam_url']
            r = requests.get(url, cookies=cookies)
            page_soup = soup(r.content, "html.parser")
            tags = page_soup.find("div", {"class":"glance_tags popular_tags"})
            tags = tags.text
            tags = tags.replace('\t', '')
            tags = tags.replace('\r', '')
            tags = tags.replace('\n', ', ')
            tags = tags.replace('+, ', '')
            tags = tags.replace(', , ', '')
            #tags.strip()
            print(tags)
            print('---------')
            df_tags = pd.DataFrame([tags])
            df_tag = df_tag.append(df_tags)
    except 
    
df_tag_2 = df_tag.reset_index()
df_tag_final = df_tag_2.drop(['index'], axis=1)

steam_tags = pd.concat([df_links, df_tag_final], axis=1, sort=False)
steam_tags.to_csv('steam_tags', sep="\t", header=['title', 'steam_url', 'steam_tags'])

FPS, Realistic, Action, Shooter, Multiplayer, Military, Tactical, Singleplayer, First-Person, War, Online Co-Op, Atmospheric, Violent, Co-op, Difficult, Gore, Open World, Massively Multiplayer, Choices Matter, Indie
---------
Action, Hunting, Co-op, Multiplayer, Open World, Third Person, RPG, Adventure, Fantasy, Singleplayer, Action RPG, Character Customization, Difficult, Great Soundtrack, Exploration, Atmospheric, Replay Value, JRPG, Hack and Slash, Souls-like
---------
FPS, Multiplayer, Shooter, Action, Team-Based, Competitive, Tactical, First-Person, e-sports, PvP, Online Co-Op, Military, Co-op, Strategy, War, Trading, Difficult, Realistic, Fast-Paced, Moddable
---------
Free to Play, Action RPG, Hack and Slash, RPG, Multiplayer, Massively Multiplayer, Action, Loot, Fantasy, Adventure, MMORPG, Indie, Dungeon Crawler, Online Co-Op, Co-op, Character Customization, PvE, Dark Fantasy, Inventory Management, Fishing
---------
Free to Play, Action, Co-op, Multiplayer, Third-Person Shooter

In [153]:
df_tag_2 = df_tag
df_tag_3 = df_tag_2.reset_index()
df_tag_final = df_tag_3.drop(['index'], axis=1)
df_tag_final

,0
0,"FPS, Realistic, Action, Shooter, Multiplayer, ..."
1,"Action, Hunting, Co-op, Multiplayer, Open Worl..."
2,"FPS, Multiplayer, Shooter, Action, Team-Based,..."
3,"Free to Play, Action RPG, Hack and Slash, RPG,..."
4,"Free to Play, Action, Co-op, Multiplayer, Thir..."
5,"RPG, Turn-Based, Co-op, Story Rich, Fantasy, O..."
6,"Open World, Action, RPG, Singleplayer, Adventu..."
7,"Survival, Crafting, Multiplayer, Open World, S..."
8,"FPS, Multiplayer, Tactical, Shooter, Action, T..."
9,"Open World, Action, Multiplayer, First-Person,..."


In [186]:
tags = " "
tags = tags.replace('\t', '')
tags = tags.replace('\r', '')
tags = tags.replace('\n', ', ')
tags = tags.replace('+, ', '')
tags = tags.replace(', , ', '')

In [187]:
tags

' '